In [54]:
import pandas as pd
from pathlib import Path

# 📍 Pfad zur CSV anpassen
csv_path = Path(r"C:\Users\asus\Desktop\Masterarbeit\Realisierte Erzeugung\Realisierte_Erzeugung_2325.csv")
xlsx_path = Path(r"C:\Users\asus\Desktop\Masterarbeit\Realisierte Erzeugung\Strommix_mit_Emissionen_2325_EE_Null.xlsx")

# 📊 Emissionsfaktoren [g CO2 / kWh] 
# in der inventarbasierten Strommix-Berechnung wird Wind, PV, Wasser- und Biomassestrom 
#i. d. R. mit 0 g CO₂ pro kWh angesetzt, weil nur die direkten Emissionen am Kraftwerksstandort gezählt werden.
#Lebenszyklus-Emissionen (LCA) – also alles, 
#was in vorgelagerten Prozessen (Herstellung von Modulen, Stahl, Beton, Transport, Rückbau …) 
#anfällt – werden dabei explizit ausgeklammert.






FACTORS_G_PER_KWH = {
    "Braunkohle [MWh]":      1049,
    "Steinkohle [MWh]":       867,
    "Erdgas [MWh]":           358,
    "Biomasse [MWh]":          0,
    "Photovoltaik [MWh]":      0,
    "Wind Onshore [MWh]":      0,
    "Wind Offshore [MWh]":     0,
    "Wasserkraft [MWh]":       0,
    "Kernenergie [MWh]":       0
}

# 📥 CSV einlesen
df = pd.read_csv(
    csv_path,
    sep=";",
    decimal=",",
    parse_dates=["Datum von", "Datum bis"],
    dayfirst=True,
    dtype=str,
)

# Spaltennamen bereinigen
df.columns = df.columns.str.strip()

# 🧮 Zahlenformate umwandeln
for col in df.columns:
    if col.endswith("[MWh]"):
        df[col] = (
            df[col]
            .str.replace(".", "", regex=False)
            .str.replace(",", ".", regex=False)
            .astype(float)
        )

# 🧪 CO2-Spalten erzeugen
for col, factor in FACTORS_G_PER_KWH.items():
    if col in df.columns:
        co2_col = col.replace("[MWh]", "[t_CO2]")
        df[co2_col] = df[col] * factor / 1000  # g → t
    else:
        print(f"⚠️ Spalte '{col}' nicht im Datensatz gefunden.")

# ➕ Summen und CO₂-Intensität
mwh_cols = [c for c in df.columns if c.endswith("[MWh]")]
co2_cols = [c for c in df.columns if c.endswith("[t_CO2]")]
df["Sum_MWh"] = df[mwh_cols].sum(axis=1)
df["Sum_t_CO2"] = df[co2_cols].sum(axis=1)
df["Mix_g_CO2_kWh"] = (df["Sum_t_CO2"] * 1_000_000) / (df["Sum_MWh"] * 1_000)

# 💾 Excel schreiben (mit OpenPyXL als Fallback)
try:
    with pd.ExcelWriter(xlsx_path, engine="xlsxwriter", datetime_format="yyyy-mm-dd hh:mm") as writer:
        df.to_excel(writer, index=False, sheet_name="Strommix")
except ModuleNotFoundError:
    print("⚠️ xlsxwriter nicht installiert. Fallback zu openpyxl.")
    df.to_excel(xlsx_path, index=False, sheet_name="Strommix")

print("✅ Excel erfolgreich geschrieben nach:", xlsx_path)


⚠️ xlsxwriter nicht installiert. Fallback zu openpyxl.
✅ Excel erfolgreich geschrieben nach: C:\Users\asus\Desktop\Masterarbeit\Realisierte Erzeugung\Strommix_mit_Emissionen_2325_EE_Null.xlsx
